In [1]:
!pip install transformers 
!pip install torch 
!pip install sklearn 
!pip install pandas as pd

  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://g

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertModel
import torch

# Load your dataframe (replace 'your_data.csv' with the actual file path)
df = pd.read_csv('/kaggle/input/laws-dataset-before-aug/Laws dataset - Domains (1).csv')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df = df.drop(df.columns[0],axis=1)
df

,Sl. No.,Intent,Entity,Description,Domain
0,1,Unauthorized Computer System Access,"Login credentials, firewall settings",Involves unauthorized individuals gaining acce...,Cyber crime
1,2,Fraudulent Online Transactions,"Credit/debit card details, transaction records",Transactions made online using stolen or clone...,Cyber crime
2,3,Identity Theft Incidents,"Personal identification information, online pr...",Stealing someone's personal information to imp...,Cyber crime
3,4,Personal Data Breaches,"Personal data (names, addresses, emails), brea...",Unauthorized access and extraction of personal...,Cyber crime
4,5,Cyberbullying or Online Harassment,"Social media accounts, chat logs","The use of digital platforms to intimidate, ha...",Cyber crime
...,...,...,...,...,...
295,296,Urban Development and Planning Law Disputes,"Development plans, zoning challenges",Handling disputes related to urban development...,Civil
296,297,Foreign Investment and FDI Regulation Disputes,"FDI policies, joint venture agreements",Addressing disputes related to foreign direct ...,Civil
297,298,E-Commerce and Online Business Disputes,"Online transaction records, consumer complaints",Resolving disputes arising from e-commerce and...,Civil
298,299,Social Media and Digital Content Disputes,"Social media policies, copyright takedown notices",Handling legal issues related to social media ...,Civil


In [4]:
# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def bert_encode(text):
    # Tokenize the text and get input IDs
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    # Generate embeddings using BERT
    with torch.no_grad():
        outputs = model(**inputs)
    # Return the embeddings of the [CLS] token
    return outputs.last_hidden_state[:, 0, :].numpy()

# Preprocess and generate embeddings
# Assuming 'Intent', 'Entity', and 'Description' columns are combined into a single text column named 'Text'
df['intent_emb'] = df['Intent'].apply(lambda x: bert_encode(x))
df['entity_emb'] = df['Entity'].apply(lambda x: bert_encode(x))
df['desc_emb'] = df['Description'].apply(lambda x: bert_encode(x))

In [5]:
df

,Sl. No.,Intent,Entity,Description,Domain,intent_emb,entity_emb,desc_emb
0,1,Unauthorized Computer System Access,"Login credentials, firewall settings",Involves unauthorized individuals gaining acce...,Cyber crime,"[[-0.3018479, 0.28632206, -0.37314367, 0.04375...","[[-0.38866845, -0.323512, -0.598392, 0.2306918...","[[-0.07622224, -0.25724956, -0.88032246, -0.21..."
1,2,Fraudulent Online Transactions,"Credit/debit card details, transaction records",Transactions made online using stolen or clone...,Cyber crime,"[[-0.24881385, 0.082826234, -0.80183595, -0.03...","[[-0.33179474, 0.08302162, -0.39112487, -0.019...","[[-0.09414861, -0.40565014, -0.6238309, 0.0991..."
2,3,Identity Theft Incidents,"Personal identification information, online pr...",Stealing someone's personal information to imp...,Cyber crime,"[[-0.6331117, 0.28610164, -1.3859601, -0.03639...","[[-0.23002872, -0.115571514, -0.49049783, 0.26...","[[0.15638226, -0.122622184, -0.88637733, -0.03..."
3,4,Personal Data Breaches,"Personal data (names, addresses, emails), brea...",Unauthorized access and extraction of personal...,Cyber crime,"[[-0.35581866, 0.078164004, -0.76763093, -0.14...","[[-0.4416205, 0.19175817, -0.5271643, 0.230069...","[[-0.2778337, 0.1530797, -1.0064421, 0.0914441..."
4,5,Cyberbullying or Online Harassment,"Social media accounts, chat logs","The use of digital platforms to intimidate, ha...",Cyber crime,"[[-0.11110083, -0.05369243, -0.92362446, -0.16...","[[0.11978668, -0.14822404, -0.24963653, -0.030...","[[-0.31064683, 0.10674395, -0.8047696, 0.11190..."
...,...,...,...,...,...,...,...,...
295,296,Urban Development and Planning Law Disputes,"Development plans, zoning challenges",Handling disputes related to urban development...,Civil,"[[-0.22259995, 0.07118839, -0.81731606, -0.334...","[[-0.56237125, -0.11687764, -0.038049333, -0.0...","[[-0.9736325, -0.2576345, -1.2218844, -0.45108..."
296,297,Foreign Investment and FDI Regulation Disputes,"FDI policies, joint venture agreements",Addressing disputes related to foreign direct ...,Civil,"[[-0.13449226, 0.11450113, -0.8037601, -0.2120...","[[-0.32282892, -0.18120492, -0.3524224, 0.1631...","[[-0.79295886, -0.07911948, -1.293942, -0.4391..."
297,298,E-Commerce and Online Business Disputes,"Online transaction records, consumer complaints",Resolving disputes arising from e-commerce and...,Civil,"[[-0.07279284, 0.067883186, -0.9709054, 0.0614...","[[-0.3230663, 0.14258994, -0.792545, -0.010953...","[[-0.8045694, -0.051980983, -0.9347986, -0.348..."
298,299,Social Media and Digital Content Disputes,"Social media policies, copyright takedown notices",Handling legal issues related to social media ...,Civil,"[[-0.2736617, 0.27722508, -1.06974, -0.0786468...","[[-0.16217251, -0.14981711, -0.39620486, -0.11...","[[-0.6139349, -0.26139605, -1.2443974, -0.3911..."


In [6]:
df.iloc[:,4:7]

,Domain,intent_emb,entity_emb
0,Cyber crime,"[[-0.3018479, 0.28632206, -0.37314367, 0.04375...","[[-0.38866845, -0.323512, -0.598392, 0.2306918..."
1,Cyber crime,"[[-0.24881385, 0.082826234, -0.80183595, -0.03...","[[-0.33179474, 0.08302162, -0.39112487, -0.019..."
2,Cyber crime,"[[-0.6331117, 0.28610164, -1.3859601, -0.03639...","[[-0.23002872, -0.115571514, -0.49049783, 0.26..."
3,Cyber crime,"[[-0.35581866, 0.078164004, -0.76763093, -0.14...","[[-0.4416205, 0.19175817, -0.5271643, 0.230069..."
4,Cyber crime,"[[-0.11110083, -0.05369243, -0.92362446, -0.16...","[[0.11978668, -0.14822404, -0.24963653, -0.030..."
...,...,...,...
295,Civil,"[[-0.22259995, 0.07118839, -0.81731606, -0.334...","[[-0.56237125, -0.11687764, -0.038049333, -0.0..."
296,Civil,"[[-0.13449226, 0.11450113, -0.8037601, -0.2120...","[[-0.32282892, -0.18120492, -0.3524224, 0.1631..."
297,Civil,"[[-0.07279284, 0.067883186, -0.9709054, 0.0614...","[[-0.3230663, 0.14258994, -0.792545, -0.010953..."
298,Civil,"[[-0.2736617, 0.27722508, -1.06974, -0.0786468...","[[-0.16217251, -0.14981711, -0.39620486, -0.11..."


In [7]:
# Function to flatten and concatenate embeddings
import numpy as np
def flatten_and_concat_embeddings(df, embedding_cols):
    concatenated_embeddings = []
    
    for _, row in df.iterrows():
        flattened_embeddings = np.hstack([np.array(row[col]).flatten() for col in embedding_cols])
        concatenated_embeddings.append(flattened_embeddings)
    
    return np.array(concatenated_embeddings)

# Columns containing embeddings
embedding_columns = ['intent_emb', 'entity_emb', 'desc_emb']

# Flatten and concatenate embeddings
X = flatten_and_concat_embeddings(df, embedding_columns)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, df['Domain'], test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Accuracy: 0.9833333333333333


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

In [9]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from keras.models import Sequential
from keras.layers import Dense, Dropout

# SVM Classifier
svm_clf = SVC()
svm_clf.fit(X_train, y_train)
svm_pred = svm_clf.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_pred)
print(f'SVM Accuracy: {svm_accuracy}')

# Random Forest Classifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
print(f'Random Forest Accuracy: {rf_accuracy}')


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

SVM Accuracy: 0.9833333333333333
Random Forest Accuracy: 0.9833333333333333


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

In [10]:
!pip install tensorflow

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical  # Use to_categorical from keras.utils
from sklearn.preprocessing import LabelEncoder

# Assuming df is your DataFrame
X = np.array(df['intent_emb'].tolist())
y = np.array(df['Domain'])

# Encoding labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_encoded_categorical = to_categorical(y_encoded)  # Use to_categorical

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded_categorical, test_size=0.2, random_state=42)


In [12]:
# LSTM Model
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [13]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10
7/7 [==============================] - 8s 152ms/step - loss: 1.0500 - accuracy: 0.6296 - val_loss: 0.9452 - val_accuracy: 1.0000
Epoch 2/10
7/7 [==============================] - 0s 11ms/step - loss: 0.8733 - accuracy: 0.9398 - val_loss: 0.7227 - val_accuracy: 1.0000
Epoch 3/10
7/7 [==============================] - 0s 11ms/step - loss: 0.6457 - accuracy: 0.9769 - val_loss: 0.5293 - val_accuracy: 1.0000
Epoch 4/10
7/7 [==============================] - 0s 11ms/step - loss: 0.4599 - accuracy: 0.9815 - val_loss: 0.3282 - val_accuracy: 1.0000
Epoch 5/10
7/7 [==============================] - 0s 10ms/step - loss: 0.3068 - accuracy: 0.9722 - val_loss: 0.2017 - val_accuracy: 1.0000
Epoch 6/10
7/7 [==============================] - 0s 10ms/step - loss: 0.2069 - accuracy: 0.9815 - val_loss: 0.1482 - val_accuracy: 1.0000
Epoch 7/10
7/7 [==============================] - 0s 10ms/step - loss: 0.1475 - accuracy: 0.9907 - val_loss: 0.1077 - val_accuracy: 1.0000
Epoch 8/10
7/7 [==========

In [14]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

2/2 [==============================] - 0s 7ms/step - loss: 0.0971 - accuracy: 0.9667
Accuracy: 0.9666666388511658


In [15]:
!pip install spacy



In [22]:
from transformers import BertTokenizer, BertModel
import torch

# BERT model and tokenizer initialization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

import spacy

# Load Spacy's small English model
nlp = spacy.load("en_core_web_sm")

# Function to extract potential entity, and treat rest as intent and description
def extract_elements(input_line):
    doc = nlp(input_line)
    entities = [ent.text for ent in doc.ents]
    entity = ' '.join(entities)  # Combine all entities
    # Remove entity from line to consider the rest as intent and description
    for ent in entities:
        input_line = input_line.replace(ent, "")
    return input_line.strip(), entity, input_line.strip()


# Function to convert text to BERT embedding
def text_to_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Function to predict the domain class
def predict_domain(input_line):
    # Extract elements from the input line
    intent, entity, description = extract_elements(input_line)
    
    # Convert to embeddings
    intent_emb = text_to_embedding(intent)
    entity_emb = text_to_embedding(entity)
    desc_emb = text_to_embedding(description)

    # Concatenate embeddings
    X_input = np.hstack([intent_emb.flatten(), entity_emb.flatten(), desc_emb.flatten()]).reshape(1, -1)
    
    # Predict using the trained model
    y_pred = clf.predict(X_input)
    return y_pred[0]

input_line = "i gave phone number to a lottery at shopping mall. they called me and asked my bank account number and password. now they stole my money and i got deceived. please help me get my money back. "
domain_class = predict_domain(input_line)
print(f"Predicted Domain Class: {domain_class}")


Predicted Domain Class: Cyber crime


In [23]:
input_line = "I parked my car at roadside at no parking zone. however i kept 10000 inside car. someone used some tricks to open the car and theft occurred. It occurred because automatic vehicle. please mae ammendment formy ;osses"
domain_class = predict_domain(input_line)
print(f"Predicted Domain Class: {domain_class}")


Predicted Domain Class: Traffic
